In [201]:
from pandas import DataFrame, read_csv
import pandas as pd

In [202]:
data = pd.read_csv('robot_walk.csv')
data.head()

,0.0,0.0.1,0.0.2,0,11.01938630231983,3.3977230072979054,3.52495742092446
0,0.0,0.0,0.0,0.0,6.346492,3.448256,3.524957
1,0.0,0.0,0.0,0.0,4.631166,3.468037,3.524957
2,0.0,0.0,0.0,0.0,3.365291,3.483078,3.524957
3,0.0,0.0,0.0,0.0,2.336458,3.495585,3.524957
4,0.0,0.0,0.0,0.0,1.673043,3.503787,3.524957


In [203]:
sensors = data.iloc[:,[0,1,2,3]]
sensors.head()

,0.0,0.0.1,0.0.2,0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [204]:
orientation = data.iloc[:,[4]]
orientation.head()

,11.01938630231983
0,6.346492
1,4.631166
2,3.365291
3,2.336458
4,1.673043


In [205]:
labels = data.iloc[:,[5,6]]
labels.head()

,3.3977230072979054,3.52495742092446
0,3.448256,3.524957
1,3.468037,3.524957
2,3.483078,3.524957
3,3.495585,3.524957
4,3.503787,3.524957


In [206]:
from sklearn import preprocessing

sensors_values = sensors.values
min_max_sensors_scaler = preprocessing.MinMaxScaler()
x_sensors_scaled = min_max_sensors_scaler.fit_transform(sensors_values)

In [207]:
orientation_values = orientation.values
min_max_orientation_scaler = preprocessing.MinMaxScaler()
x_orientation_scaled = min_max_orientation_scaler.fit_transform(orientation_values)

In [208]:
labels_values = labels.values
min_max_labels_scaler = preprocessing.MinMaxScaler()
labels_scaled = min_max_labels_scaler.fit_transform(labels_values)

In [209]:
import numpy as np
X = np.c_[x_sensors_scaled, x_orientation_scaled]

In [212]:

from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, labels_scaled, test_size=0.2)

In [220]:
import random

from scipy.stats import reciprocal
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network.multilayer_perceptron import MLPRegressor

neural_network = MLPRegressor(activation='relu', solver='lbfgs', early_stopping=True)

params = {
	"hidden_layer_sizes": [(random.randint(0, 100), random.randint(0, 100)) for k in range(10) ]
	}

grid_search = GridSearchCV(neural_network, param_grid=params, cv=5, return_train_score=True, n_jobs=1, verbose=2)

scores = grid_search.fit(X_train, y_train)
scores.cv_results_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] hidden_layer_sizes=(19, 81) .....................................
[CV] ...................... hidden_layer_sizes=(19, 81), total=   0.4s
[CV] hidden_layer_sizes=(19, 81) .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ...................... hidden_layer_sizes=(19, 81), total=   0.3s
[CV] hidden_layer_sizes=(19, 81) .....................................
[CV] ...................... hidden_layer_sizes=(19, 81), total=   0.4s
[CV] hidden_layer_sizes=(19, 81) .....................................
[CV] ...................... hidden_layer_sizes=(19, 81), total=   0.4s
[CV] hidden_layer_sizes=(19, 81) .....................................
[CV] ...................... hidden_layer_sizes=(19, 81), total=   0.4s
[CV] hidden_layer_sizes=(59, 40) .....................................
[CV] ...................... hidden_layer_sizes=(59, 40), total=   0.4s
[CV] hidden_layer_sizes=(59, 40) .....................................
[CV] ...................... hidden_layer_sizes=(59, 40), total=   0.4s
[CV] hidden_layer_sizes=(59, 40) .....................................
[CV] ...................... hidden_layer_sizes=(59, 40), total=   0.4s
[CV] hidden_layer_sizes=(59, 40) .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   31.7s finished


{'mean_fit_time': array([0.3714787 , 0.38597693, 0.44002752, 0.63874059, 0.76346679,
        0.65768785, 1.53366394, 0.9221385 , 0.24861164, 0.33425989]),
 'std_fit_time': array([0.03791899, 0.03568725, 0.03273294, 0.00933622, 0.07443381,
        0.13607494, 0.19740117, 0.10498092, 0.27312034, 0.17407248]),
 'mean_score_time': array([0.00067086, 0.00062957, 0.00061188, 0.00060539, 0.00067654,
        0.00077024, 0.00259004, 0.00070124, 0.00066862, 0.00062246]),
 'std_score_time': array([1.11927242e-04, 8.91978170e-05, 7.05728015e-05, 6.08672873e-06,
        8.43999884e-05, 1.97157640e-04, 3.16186317e-03, 8.28843116e-05,
        7.14017432e-05, 9.42389237e-05]),
 'param_hidden_layer_sizes': masked_array(data=[(19, 81), (59, 40), (47, 63), (86, 64), (69, 74),
                    (13, 100), (64, 77), (29, 68), (84, 1), (30, 2)],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),

In [221]:
scores.best_estimator_


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(47, 63), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [222]:
scores.best_score_

0.9166152271095046

In [232]:
from sklearn.metrics import explained_variance_score

predictions = grid_search.predict(X_validation)

In [233]:
precision = explained_variance_score(predictions, y_validation)

In [234]:
precision

0.9501318845047286

In [235]:
import pickle
pickle.dump(grid_search, open('model.sav', 'wb'))

In [240]:
pickle.dump(min_max_labels_scaler, open('labels_scaler', 'wb'))
pickle.dump(min_max_orientation_scaler, open('orientation_scaler', 'wb'))
pickle.dump(min_max_sensors_scaler, open('sensors_scaler', 'wb'))